# <img align="left" src="./images/film_strip_vertical.png"     style=" width:40px;  " > Practice lab: Deep Learning for Content-Based Filtering

在这个练习中，您将使用神经网络实现基于内容的过滤器，为电影构建一个推荐系统

# Outline <img align="left" src="./images/film_reel.png"     style=" width:40px;  " >
- [ 1 - Packages](#1)
- [ 2 - Movie ratings dataset](#2)
  - [ 2.1 Content-based filtering with a neural network](#2.1)
  - [ 2.2 Preparing the training data](#2.2)
- [ 3 - Neural Network for content-based filtering](#3)
  - [ 3.1 Predictions](#3.1)
    - [ Exercise 1](#ex01)
- [ 4 - Congratulations!](#4)


<a name="1"></a>
## 1 - Packages <img align="left" src="./images/movie_camera.png"     style=" width:40px;  ">
We will use familiar packages, NumPy, TensorFlow and helpful routines from [scikit-learn](https://scikit-learn.org/stable/). We will also use [tabulate](https://pypi.org/project/tabulate/) to neatly print tables and [Pandas](https://pandas.pydata.org/) to organize tabular data.

In [3]:
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils import *
pd.set_option("display.precision", 1)

<a name="2"></a>
## 2 - Movie ratings dataset <img align="left" src="./images/film_rating.png" style=" width:40px;" >
The data set is derived from the [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/) dataset. 

[F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>]

原始数据集包含 9000 部电影，由 600 个用户进行评分，评分在 0.5 到 5 的范围内，以 0.5 为步长增加。数据集已经缩小，以便关注于自 2000 年以来的电影和流行的流派。缩小后的数据集具有 $n_u = 395$ 个用户和 $n_m = 694$ 部电影。对于每部电影，数据集提供了电影标题、发行日期和一个或多个流派。例如，“玩具总动员3”是在2010年上映的，有几个流派：“冒险|动画|儿童|喜剧|幻想|IMAX”。除了他们的评分之外，这个数据集几乎没有关于用户的信息。这个数据集用于为下面描述的神经网络创建训练向量。

<a name="2.1"></a>
### 2.1 Content-based filtering with a neural network

在协同过滤实验中，您生成了两个向量，一个用户向量和一个项目/电影向量，它们的点积可以预测评分。这些向量仅从评分中派生而来。

基于内容的过滤也生成用户和电影特征向量，但是识别出可能存在有关用户和/或电影的其他信息可以提高预测的准确性。这些额外的信息提供给神经网络，然后如下所示生成用户和电影向量。
<figure>
    <center> <img src="./images/RecSysNN.png"   style="width:500px;height:280px;" ></center>
</figure>
提供给网络的电影内容是原始数据和一些“特征工程”的组合。回想一下第一门课程第二周第四个实验室的特征工程讨论和实验。原始特征是电影发行的年份和作为独热向量呈现的电影流派。有14种流派。工程特征是从用户评分中派生的平均评分。具有多种流派的电影每种流派都有一个训练向量。

用户内容仅由工程特征组成。每个用户计算每种流派的平均评分。此外，用户ID、评分计数和评分平均值可用，但不包括在培训或预测内容中。它们在解释数据方面非常有用。

训练集包括数据集中用户进行的所有评分。将用户和电影/项目向量一起作为训练集呈现给上述网络。对于由用户评分的所有电影，用户向量是相同的。

接下来，让我们加载并显示一些数据。

In [4]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
scaledata = True  # applies the standard scalar to data if true
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 58187



一些用户和项目/电影特征在训练中没有使用。下面，方括号“[]”中的特征，如“用户ID”，“评分计数”和“评分平均值”在训练和使用模型时不包括。请注意，对于所有评分的电影，用户向量是相同的。

In [5]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9


In [6]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8798,2004,3.8,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [7]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  4.  4.  3.5 3.5]



如上所述，我们可以看到电影6874是一部在2003年上映的动作电影。用户2平均评分动作电影为3.9分。此外，电影6874还列在犯罪和惊悚类型中。MovieLens用户给这部电影的平均评分为4分。一个训练示例由两个表中的一行和y_train中的一个评分组成。

<a name="2.2"></a>
### 2.2 Preparing the training data
回想一下第1门课程第2周，您探索了特征缩放作为提高收敛性的手段。我们将使用 [scikit learn StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)的StandardScaler对输入特征进行缩放。这在第1门课程第2周的实验室5中使用过。下面还显示了inverse_transform以生成原始输入。

In [8]:
# scale training data
if scaledata:
    item_train_save = item_train
    user_train_save = user_train

    scalerItem = StandardScaler()
    scalerItem.fit(item_train)
    item_train = scalerItem.transform(item_train)

    scalerUser = StandardScaler()
    scalerUser.fit(user_train)
    user_train = scalerUser.transform(user_train)

    print(np.allclose(item_train_save, scalerItem.inverse_transform(item_train)))
    print(np.allclose(user_train_save, scalerUser.inverse_transform(user_train)))

True
True


为了让我们能够评估结果，我们将按照第2门课程第3周讨论的方法将数据分为训练集和测试集。在这里，我们将使用 [sklean train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 来拆分和洗牌数据。请注意，将初始随机状态设置为相同的值可以确保项目、用户和y被相同方式洗牌。

In [9]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test  data shape: {item_test.shape}")

movie/item training data shape: (46549, 17)
movie/item test  data shape: (11638, 17)



经过缩放和洗牌的数据现在具有零均值。

In [10]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
1,0,0.6,0.7,0.6,0.6,0.7,0.7,0.5,0.7,0.2,0.3,0.3,0.5,0.5,0.8,0.5
0,0,1.6,1.5,1.7,0.9,1.0,1.4,0.8,-1.2,1.2,1.2,1.6,0.9,1.4,1.2,1.0
0,0,0.8,0.6,0.7,0.5,0.6,0.6,0.3,-1.2,0.7,0.8,0.9,0.6,0.2,0.6,0.6
1,0,-0.1,0.2,-0.1,0.3,0.7,0.3,0.2,1.0,-0.5,-0.7,-2.1,0.5,0.7,0.3,0.0
-1,0,-1.3,-0.8,-0.8,0.1,-0.1,-1.1,-0.9,-1.2,-1.5,-0.6,-0.5,-0.6,-0.9,-0.4,-0.9



使用MinMaxScaler对目标评级进行缩放，将目标评级缩放到-1到1之间。我们使用scikit-learn是因为它具有inverse_transform。 [scikit learn MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [11]:
scaler = MinMaxScaler((-1, 1))
scaler.fit(y_train.reshape(-1, 1))
ynorm_train = scaler.transform(y_train.reshape(-1, 1))
ynorm_test = scaler.transform(y_test.reshape(-1, 1))
print(ynorm_train.shape, ynorm_test.shape)

(46549, 1) (11638, 1)


<a name="3"></a>
## 3 - Neural Network for content-based filtering
现在，让我们按照上图所述构建一个神经网络。它将具有通过点积组合的两个网络。您将构建这两个网络。在本例中，它们将是相同的。请注意，这些网络不需要相同。如果用户内容远大于电影内容，您可能会决定相对于电影网络增加用户网络的复杂性。在这种情况下，内容相似，因此网络是相同的。

- 使用Keras顺序模型
    - 第一层是具有256个单元的密集层和relu激活。
    - 第二层是具有128个单元的密集层和relu激活。
    - 第三层是具有num_outputs个单元的密集层和线性或无激活。
网络的其余部分将提供。提供的代码不使用Keras顺序模型，而是使用Keras功能API。此格式允许更灵活地连接组件。 [functional api](https://keras.io/guides/functional_api/).


In [12]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###   
    tf.keras.layers.Dense(256,activation = 'relu',name = 'l1'),
    tf.keras.layers.Dense(128,activation = 'relu',name = 'l2'),
    tf.keras.layers.Dense(num_outputs,activation = 'linear',name = 'l3')
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
    tf.keras.layers.Dense(256,activation = 'relu',name = 'l1'),
    tf.keras.layers.Dense(128,activation = 'relu',name = 'l2'),
    tf.keras.layers.Dense(num_outputs,activation = 'linear',name = 'l3')
    ### END CODE HERE ###  
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 14)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 sequential (Sequential)        (None, 32)           40864       ['input_1[0][0]']                
                                                                                                  
 sequential_1 (Sequential)      (None, 32)           41376       ['input_2[0][0]']                
                                                                                              

In [13]:
# Public tests
from public_tests import *
test_tower(user_NN)
test_tower(item_NN)

All tests passed!
All tests passed!


<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
  You can create a dense layer with a relu activation as shown.
    
```python     
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),

    
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),

    
    ### END CODE HERE ###  
])
```    
<details>
    <summary><font size="2" color="darkblue"><b> Click for solution</b></font></summary>
    
```python 
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])
```
</details>
</details>

    


We'll use a mean squared error loss and an Adam optimizer.

In [14]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [15]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], ynorm_train, epochs=30)

Epoch 1/30
1455/1455 [==============================] - 3s 1ms/step - loss: 0.1250
Epoch 2/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1183
Epoch 3/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1167
Epoch 4/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1150
Epoch 5/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1137
Epoch 6/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1124
Epoch 7/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1118
Epoch 8/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1111
Epoch 9/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1102
Epoch 10/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1099
Epoch 11/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1090
Epoch 12/30
1455/1455 [==============================] - 2s 1ms/step - loss: 0.1081
E


评估模型以确定在测试数据上的损失。它与训练损失相当，表明该模型没有显着过度拟合训练数据。

In [16]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], ynorm_test)

364/364 [==============================] - 0s 840us/step - loss: 0.1054


0.10536185652017593

<a name="3.1"></a>
### 3.1 Predictions

下面，您将使用模型在许多情况下进行预测。

#### 预测新用户

首先，我们将创建一个新用户，并让模型为该用户建议电影。在尝试这个示例用户内容之后，可以随意更改用户内容以匹配自己的喜好，并查看模型建议的内容。请注意，评级在0.5和5.0之间，包括半步增量。

In [17]:
new_user_id = 5000
new_rating_ave = 1.0
new_action = 1.0
new_adventure = 1
new_animation = 1
new_childrens = 1
new_comedy = 5
new_crime = 1
new_documentary = 1
new_drama = 1
new_fantasy = 1
new_horror = 1
new_mystery = 1
new_romance = 5
new_scifi = 5
new_thriller = 1
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])



让我们看看新用户的评分最高的电影。回想一下，用户矢量具有偏向喜剧和浪漫的流派。下面，我们将使用一组电影/项目向量item_vecs，其中每个电影在训练/测试集中都有一个向量。这与上面的用户向量匹配，并使用缩放的向量来预测新用户对所有电影的评分。

In [18]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale the vectors and make predictions for all movies. Return results sorted by rating.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs,  item_vecs, model, u_s, i_s, 
                                                                       scaler, scalerUser, scalerItem, scaledata=scaledata)

print_pred_movies(sorted_ypu, sorted_user, sorted_items, movie_dict, maxcount = 10)

59/59 [==============================] - 0s 813us/step


y_p,movie id,rating ave,title,genres
4.81767,8622,3.48649,Fahrenheit 9/11 (2004),Documentary
4.81394,8464,3.51,Super Size Me (2004),Comedy|Documentary|Drama
4.80988,34072,3.55556,"March of the Penguins (Marche de l'empereur, La) (2005)",Documentary
4.80349,45950,3.57692,"Inconvenient Truth, An (2006)",Documentary
4.792,36708,3.70833,Family Guy Presents Stewie Griffin: The Untold Story (2005),Adventure|Animation|Comedy
4.7895,5785,3.5,Jackass: The Movie (2002),Action|Comedy|Documentary
4.78766,46976,3.69318,Stranger than Fiction (2006),Comedy|Drama|Fantasy|Romance
4.78574,8784,3.70833,Garden State (2004),Comedy|Drama|Romance
4.78463,47610,3.77907,"Illusionist, The (2006)",Drama|Fantasy|Mystery|Romance
4.77863,47997,3.72222,Idiocracy (2006),Adventure|Comedy|Sci-Fi|Thriller



如果您在上面创建了一个用户，则值得注意的是，该网络是训练为在给定包含用户流派评分的集合的用户向量的情况下预测用户评分的。如果没有具有类似评分集的用户，则仅为单个流派提供最高评级和其余最低评级可能对网络没有意义。

#### Predictions for an existing user.

让我们看看“用户36”的预测评分，这是数据集中的一个用户。我们可以将预测的评分与模型的评分进行比较。请注意，具有多个流派的电影在训练数据中出现多次。例如，“时光机器”有三种流派：冒险，动作，科幻。

In [19]:
uid =  36 
# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, scalerUser.inverse_transform(user_train), item_vecs, user_to_genre)

# scale the vectors and make predictions for all movies. Return results sorted by rating.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler, 
                                                                      scalerUser, scalerItem, scaledata=scaledata)
sorted_y = y_vecs[sorted_index]

#print sorted predictions
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, item_features, ivs, uvs, movie_dict, maxcount = 10)

59/59 [==============================] - 0s 914us/step


y_p,y,user,user genre ave,movie rating ave,title,genres
3.1,3.0,36,3.00,2.86,"Time Machine, The (2002)",Action
3.0,3.0,36,3.00,2.86,"Time Machine, The (2002)",Adventure
3.0,3.0,36,3.00,2.86,"Time Machine, The (2002)",Sci-Fi
2.0,2.0,36,1.75,3.52,Gangs of New York (2002),Crime
2.0,1.5,36,1.75,3.52,Road to Perdition (2002),Crime
1.9,1.0,36,1.50,4.00,"Beautiful Mind, A (2001)",Drama
1.6,1.0,36,1.00,4.00,"Beautiful Mind, A (2001)",Romance
1.4,1.5,36,1.50,3.52,Road to Perdition (2002),Drama
1.4,2.0,36,1.50,3.52,Gangs of New York (2002),Drama


#### Finding Similar Items

上面的神经网络生成两个特征向量，一个是用户特征向量$v_u$，一个是电影特征向量$v_m$。这些是32个条目向量，其值很难解释。但是，类似的项目将具有相似的向量。可以使用此信息进行推荐。例如，如果用户高度评价了“玩具总动员3”，可以通过选择具有相似电影特征向量的电影来推荐类似的电影。

A similarity measure is the squared distance between the two vectors $ \mathbf{v_m^{(k)}}$ and $\mathbf{v_m^{(i)}}$ :
$$\left\Vert \mathbf{v_m^{(k)}} - \mathbf{v_m^{(i)}}  \right\Vert^2 = \sum_{l=1}^{n}(v_{m_l}^{(k)} - v_{m_l}^{(i)})^2\tag{1}$$

<a name="ex01"></a>
### Exercise 1

Write a function to compute the square distance.

In [20]:
# GRADED_FUNCTION: sq_dist
# UNQ_C2
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """
    ### START CODE HERE ###     
    d = np.sum((a-b)**2)
    ### END CODE HERE ###     
    return (d)

In [21]:
# Public tests
test_sq_dist(sq_dist)

All tests passed!


In [22]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1)}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2)}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3)}")

squared distance between a1 and b1: 0.0
squared distance between a2 and b2: 0.030000000000000054
squared distance between a3 and b3: 2


<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
  While a summation is often an indication a for loop should be used, here the subtraction can be element-wise in one statement. Further, you can utilized np.square to square, element-wise, the result of the subtraction. np.sum can be used to sum the squared elements.
    
</details>

    



可以在模型训练时计算一次电影之间的距离矩阵，然后在不重新训练的情况下用于新的推荐。一旦训练了模型，第一步是获得每个电影的电影特征向量$v_m$。为此，我们将使用训练好的item_NN并构建一个小模型，以允许我们将电影向量通过它运行以生成$v_m$。

In [23]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = Model(input_item_m, vm_m)                                
model_m.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 16)]              0         
                                                                 
 sequential_1 (Sequential)   (None, 32)                41376     
                                                                 
 tf.math.l2_normalize_2 (TFO  (None, 32)               0         
 pLambda)                                                        
                                                                 
Total params: 41,376
Trainable params: 41,376
Non-trainable params: 0
_________________________________________________________________



有了电影模型，您可以通过使用模型来预测一组项目/电影向量作为输入来创建一组电影特征向量。item_vecs是所有电影向量的集合。请记住，同一部电影将作为其每种类型的单独向量出现。必须缩放它以与训练好的模型一起使用。预测的结果是每个电影的32个条目的特征向量。

In [24]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

59/59 [==============================] - 0s 707us/step
size of all predicted movie feature vectors: (1883, 32)


现在让我们计算每个电影特征向量与所有其他电影特征向量之间的平方距离矩阵：
<figure>
    <left> <img src="./images/distmatrix.PNG"   style="width:400px;height:225px;" ></center>
</figure>

然后，我们可以通过在每行中找到最小值来找到最接近的电影。我们将利用[numpy masked arrays](https://numpy.org/doc/1.21/user/tutorial-ma.html) 来避免选择相同的电影。对角线上的掩码值不会包含在计算中。

In [25]:
count = 50
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    genre1,_  = get_item_genre(item_vecs[i,:], ivs, item_features)
    genre2,_  = get_item_genre(item_vecs[min_idx,:], ivs, item_features)

    disp.append( [movie_dict[movie1_id]['title'], genre1,
                  movie_dict[movie2_id]['title'], genre2]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=[".1f", ".1f", ".0f", ".2f", ".2f"])
table

movie1,genres,movie2,genres
Save the Last Dance (2001),Drama,John Q (2002),Drama
Save the Last Dance (2001),Romance,Saving Silverman (Evil Woman) (2001),Romance
"Wedding Planner, The (2001)",Comedy,"Sweetest Thing, The (2002)",Comedy
"Wedding Planner, The (2001)",Romance,Mr. Deeds (2002),Romance
Hannibal (2001),Horror,Final Destination 2 (2003),Horror
Hannibal (2001),Thriller,"Sum of All Fears, The (2002)",Thriller
Saving Silverman (Evil Woman) (2001),Comedy,Cats & Dogs (2001),Comedy
Saving Silverman (Evil Woman) (2001),Romance,Mona Lisa Smile (2003),Romance
Down to Earth (2001),Comedy,Joe Dirt (2001),Comedy
Down to Earth (2001),Fantasy,"Haunted Mansion, The (2003)",Fantasy


The results show the model will suggest a movie from the same genre.

<a name="4"></a>
## 4 - Congratulations! <img align="left" src="./images/film_award.png" style=" width:40px;">
You have completed a content-based recommender system.    

This structure is the basis of many commercial recommender systems. The user content can be greatly expanded to incorporate more information about the user if it is available.  Items are not limited to movies. This can be used to recommend any item, books, cars or items that are similar to an item in your 'shopping cart'.